In [35]:
from mlx_lm import load, generate
import datetime
import random

model, tokenizer = load("mlx-community/Qwen2.5-14B-Instruct-1M-4bit") #mlx-community/Qwen2.5-7B-Instruct-1M-4bit")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.96G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.26k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

In [39]:
def temperature_tool(location: str):
    """
    The tool provides the current temperature for a given location (city name). The location needs to be an actual city name and not a [placeholder value]
    
    Args:
        location: The name of the city whose temperature to obtain
    Returns:
        The temperature of the requested city
    """
    return random.uniform(-5, 20)

def heating_tool(room: str, temperature: float):
    """
    The tool sets the temperature for a given roomname. The response is an indication whether the change of temperature was successful or not
    
    Args:
        room: The name of the room where to set the temperature
        temperature: The desired temperature in degree celsius
    Returns:
        a boolean indicating whether setting the temperature was succesfull or not
    """
    return { 'temperature': temperature, 'room': room, 'temperature_changed_successfully': True }

def current_time():
    """Get the current local time as a string."""
    return str(datetime.now())

tools = [temperature_tool, heating_tool, current_time]

messages = [
    {"role": "system", "content": """
    You are an AI assistant specialized in tool-based problem solving. 
    Your expertise lies in analyzing questions to determine whether to use specialized tools or provide direct knowledge-based responses, 
    ensuring optimal resource utilization. If you are lacking information from the user, ask for further information before making a decision. 
    Never make up any data for your responses but use the provided information only unless it is common knowledge.
    Never fill in missing data you need from the customer unless explicitly being asked for it.
    Format your response as XML with the following format:
    <answer>
     <response>[your answer or question or tool invocation]</response>
     <tool_call>[if applicable: what tool to call and with which parameters]</tool_call>
     <completed>[true is the conversation is done, or false if further customer responses are needed]</completed>
    </answer>
    """},
    {"role": "user", "content": "change the temperature"},
    {"role": "assistant", "content": "<answer>"}
]
prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=False, tools=tools, tokenize=False, continue_final_message=True
)

response = generate(model, tokenizer, prompt=prompt, verbose=True)
response


 <response>Could you please specify the room name and the desired temperature in degrees Celsius?</response>
 <completed>false</completed>
</answer>
Prompt: 523 tokens, 100.337 tokens-per-sec
Generation: 31 tokens, 10.865 tokens-per-sec
Peak memory: 12.597 GB


'\n <response>Could you please specify the room name and the desired temperature in degrees Celsius?</response>\n <completed>false</completed>\n</answer>'

In [40]:
messages[-1]['content'] += response
messages.append({"role": "user", "content": 'in the bedroom'})
messages.append({"role": "assistant", "content": "<answer>"})

prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=False, tools=tools, continue_final_message=True
)

response = generate(model, tokenizer, prompt=prompt, verbose=True)
response


 <response>What is the desired temperature for the bedroom in degrees Celsius?</response>
 <completed>false</completed>
</answer>
Prompt: 568 tokens, 103.614 tokens-per-sec
Generation: 28 tokens, 10.759 tokens-per-sec
Peak memory: 12.597 GB


'\n <response>What is the desired temperature for the bedroom in degrees Celsius?</response>\n <completed>false</completed>\n</answer>'

In [41]:
messages[-1]['content'] += response
messages.append({"role": "user", "content": 'to 18 degree'})
messages.append({"role": "assistant", "content": "<answer>"})

prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=False, tools=tools, continue_final_message=True
)

response = generate(model, tokenizer, prompt=prompt, verbose=True)
response


 <response>Setting the temperature for the bedroom to 18 degrees Celsius.</response>
 <completed>false</completed>
<tool_call>
{"name": "heating_tool", "arguments": {"room": "bedroom", "temperature": 18}}
</tool_call>
</answer>
Prompt: 612 tokens, 100.432 tokens-per-sec
Generation: 58 tokens, 10.811 tokens-per-sec
Peak memory: 12.597 GB


'\n <response>Setting the temperature for the bedroom to 18 degrees Celsius.</response>\n <completed>false</completed>\n<tool_call>\n{"name": "heating_tool", "arguments": {"room": "bedroom", "temperature": 18}}\n</tool_call>\n</answer>'

In [42]:
messages[-1]['content'] += response
messages.append({"role": "tool", "name": "heating_tool", "content": "true"})
messages.append({"role": "assistant", "content": "<answer>"})

prompt = tokenizer.apply_chat_template(
    messages, add_generation_prompt=False, tools=tools, continue_final_message=True
)

response = generate(model, tokenizer, prompt=prompt, verbose=True)
response


 <response>The temperature for the bedroom has been successfully set to 18 degrees Celsius.</response>
 <completed>true</completed>
</answer>
Prompt: 691 tokens, 101.970 tokens-per-sec
Generation: 31 tokens, 10.934 tokens-per-sec
Peak memory: 12.597 GB


'\n <response>The temperature for the bedroom has been successfully set to 18 degrees Celsius.</response>\n <completed>true</completed>\n</answer>'